Task1_0620. 네이버 영화 순위 사이트에서 영화제목, 가격, 타입(구매 or 대여) 정보를 가져와서 TITLE, PRICE, TYPE 3개의 컬럼과 100개의 데이터포인트로 구성된 데이터프레임을 출력하세요.

In [8]:
# 영화에서 제목 가져오기
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

# HTTP 요청에서 사용될 헤더 정보를 설정합니다
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}

url = "https://serieson.naver.com/v3/movie/ranking/realtime"

response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
else:
    print('response failed')

titles = soup.find_all('span', class_='Title_title__s9o0D')
types = soup.find_all('span', class_='Price_text__pRk_f')
prices = soup.find_all('span', class_='Price_price__GqXqo')
# print(titles)
# print(types)
# print(prices)

# ti={}
# pattern = re.compile(r'^[^\(]+')
ti = {}
ty = {}
pr = {}
for idx, title in enumerate(titles):
    ti[idx+1] = title.text
    # print(ti)
for idx, type in enumerate(types):
    ty[idx+1] = type.text
for idx, prices in enumerate(prices):
    pr[idx+1] = prices.text
# print(ti)
# print(ty)
# print(pr)


def merge_dicts(*dicts):
    merge_dict = {}
    for d in dicts:
        for key, value in d.items():
            if key in merge_dict:
                # print(key, value)
                current_value = merge_dict[key]
                if isinstance(current_value, list):
                    if not value in current_value:
                        current_value.append(value)
                elif current_value != value:
                    merge_dict[key] = [current_value, value]
            else:
                merge_dict[key] = value

    return merge_dict   

result = merge_dicts(ti, ty, pr)
# print(result)

df = pd.DataFrame(result).T
# df.reindex
df.rename(columns={0: 'Title', 1: 'Type', 2: 'Price'}, inplace=True)
df

,Title,Type,Price
1,인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정),구매,"7,150캐시"
2,콩나물,구매,"1,000캐시"
3,태극기 휘날리며 - 특별판,구매,"10,000캐시"
4,설계자,대여,"11,000캐시"
5,원피스 필름 레드,대여,"1,540캐시"
...,...,...,...
82,마스크,대여,"1,300캐시"
83,사운드 오브 뮤직,구매,"5,500캐시"
84,마지막 한 걸음까지,구매,500캐시
85,흑인 오르페,구매,500캐시


In [7]:
# 강사님 버전
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# URL 설정
url = 'https://serieson.naver.com/v3/movie/ranking/realtime'

# HTTP GET 요청
response = requests.get(url)
response.raise_for_status()  # 요청이 성공했는지 확인

# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(response.content, 'html.parser')

# 영화 제목과 가격을 저장할 리스트
titles = []
prices = []
types = []

# 영화 제목과 가격 추출
movies = soup.select('.RankingList_ranking_list__N4QsH > li')

for movie in movies:
    title_tag = movie.select_one('.Title_title__s9o0D')
    if title_tag:
        title = title_tag.get_text(strip=True)
    
        # 가격 및 타입 추출
        price_items = movie.select('.Price_price_item__tOOqb')
        for price_item in price_items:
            type_tag = price_item.select_one('.Price_text__pRk_f')
            price_tag = price_item.select_one('.Price_price__GqXqo')
            if type_tag and price_tag:
                type_text = type_tag.get_text(strip=True)
                price = price_tag.get_text(strip=True)
                
                # 가격에서 "캐시" 및 SVG 태그 제거
                price = re.sub(r'캐시.*', '', price).strip()
                
                titles.append(title)
                prices.append(price)
                types.append(type_text)

# 데이터프레임 생성
df = pd.DataFrame({
    'Title': titles,
    'Price': prices,
    'Type': types
})

# 데이터프레임 출력
df


,Title,Price,Type
0,인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정),"7,150",구매
1,콩나물,"1,000",구매
2,태극기 휘날리며 - 특별판,"10,000",구매
3,설계자,"11,000",대여
4,원피스 필름 레드,"1,540",대여
...,...,...,...
81,마스크,"1,300",대여
82,사운드 오브 뮤직,"5,500",구매
83,마지막 한 걸음까지,500,구매
84,흑인 오르페,500,구매



Task2_0620. url = 'https://news.daum.net/politics#1'은 정치기사 1페이지인데 10페이지에 있는 기사를 모두 출력하세요. 

In [23]:
from bs4 import BeautifulSoup
import requests as rq
import re
import pandas as pd

# HTTP 요청에서 사용될 헤더 정보를 설정합니다
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}

urls = []
for i in range(1, 11):
    # print(i)
    # url = f'https://news.daum.net/politics#{i}'
    url = f'https://news.daum.net/breakingnews/politics?page={i}'
    # urls.append(url)
    print(url)
    r = rq.get(url, headers=headers)
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')

    # lines = soup.select('body > div > main > section > div > div > ul > li > div > div > strong > a')
    lines = soup.select('.tit_thumb')
    lines[:13]
    # 한글로만 가져올때
    for i, tag in enumerate(lines, start=1):
        text = tag.get_text().strip().replace(',', '')
        matches = re.findall('[가-힣0-9]+', text)
        print(f'{i}. {" ".join(matches)}')
# print(urls)







https://news.daum.net/breakingnews/politics?page=1
1. 법사위 채 해병 특검 입법청문회 이종섭 임성근 출석 요구 09 45
2. 국민통합위 27일 국민통합 컨퍼런스 개최 서울경제 09 44
3. 성남시 내달부터 우울 불안 시민에 심리상담 지원 연합뉴스 09 44
4. 속보 유승민 대표 경선 불출마 무의미한 도전 세계일보 09 44
5. 포토 최고위 마친 이재명 대표 아이뉴스24 09 43
6. 포토 회의실 나서는 이재명 아이뉴스24 09 43
7. 전대 불붙는데 어대명 민주당 흥행카드가 없다 이런정치 헤럴드경제 09 43
8. 전격시사 이광수 광수네복덕방 대표 정치권 논쟁 중 종합부동산세 폐지 존치 09 43
9. 탈북민단체 어젯밤 대북 전단 30만 장 살포 09 43
10. 정청래 법사위원장 응원하는 박찬대 원내대표 연합뉴스 09 43
11. 정청래 법사위원장 응원하는 박찬대 원내대표 연합뉴스 09 42
12. 정청래 법사위원장 응원하는 박찬대 원내대표 연합뉴스 09 42
13. 속보 합참 북한군 여러명 침범 경고사격 이후 북상 조선비즈 09 42
14. 원희룡 한동훈 23일 국회서 나란히 출마선언 나경원은 언제 머니투데이 09 42
15. 박찬대 해병대원 수사 외압 정점에 특검법 거부는 범죄 자백 뉴스1 09 42
16. 전대 불붙는데 어대명 민주당 흥행카드가 없다
17. 이광수 광수네복덕방 대표 정치권 논쟁 중 종합부동산세 폐지 존치
18. 카드 만들면 최대 35만원 풍차돌리기 에 한 달 10명 가입하고 7명 탈퇴한다
19. 채권시장 자본조달 러시 채권개미 지금이 기회
https://news.daum.net/breakingnews/politics?page=2
1. 국방장관 방사청장 폴란드 50 운용 점검 유지보수 협력 09 42
2. 원희룡 한동훈과 한날 출사표 23일 국회서 대표 출마 선언 매일경제 09 42
3. 대화하는 이재명 대표와 서영교 최고위원 뉴스1 09 41
4. 최고위회의 모두발언하는 이재명 대표 

In [12]:
#task2 coding by 김성현
import requests
from bs4 import BeautifulSoup
import re

for j in range(1, 11):
    #https://news.daum.net/breakingnews/politics?page=1
    url = f'https://news.daum.net/breakingnews/politics?page={j}'
    print(url)
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    lines = soup.find_all('a',class_="link_txt")
    lines = lines[7:-21]
    for i, tag in enumerate(lines, 1):
        text = tag.get_text().strip().replace(',', '')
        matches = re.findall('[가-힣0-9]+', text)
        print(f"{i}. {' '.join(matches)}")
    print(f'페이지수: {j}')

https://news.daum.net/breakingnews/politics?page=1
1. 북 러 군사 협력 범위 불분명 생각보다 더 장기적 더 위험
2. 경기도 무장애 관광지 컨설팅 사업 참여 관광지 모집
3. 정유라 조민 대법 판결 남았는데 이 타이밍에 결혼을
4. 전략사령부 올 하반기 창설 사령관은 육 해 공군 장성 순환보직
5. 게시판 식약처 의약품안전관리원 약물안전캠페인
6. 원희룡 23일 국회서 당대표 출마 선언 한동훈과 같은날
7. 유승민 전당대회 불출마 무의미한 도전
8. 윤상현 한동훈 사석서 그 사람 이라고 신뢰 관계 바닥
9. 김현기 서울시의장 몽골 울란바토르에 소방차 10대 전달
10. 탈북단체 대북전단 30만장 살포
11. 북한 6 25 미제 반대 투쟁의 날 앞두고 원수 격멸 의지 굳건
12. 현대 폴란드 국제해양 안보포럼 서 잠수함 플랫폼 제안
13. 유승민 국민의힘 당대표 경선 불출마 절박함이 시작될 때 역할 하겠다 상보
14. 박찬대 건희위원회로 전락한 권익위 고무줄 청탁금지법 지적
15. 대통령실 야 단독소집 운영위 불참키로 야 증인 채택할듯
페이지수: 1
https://news.daum.net/breakingnews/politics?page=2
1. 속보 무의미한 도전이라 결론 유승민 당대표 불출마
2. 속보 유승민 전당대회 불출마 무의미한 도전
3. 원희룡 한동훈과 같은 날 같은 곳에서 당대표 출마 선언
4. 원희룡 23일 국회서 출마선언 과 1시간 간격 같은 날 같은 장소
5. 우크라 무기지원은 러 행동에 달려 155 포탄 등 방산 대기중
6. 박찬대 오늘 채해병 특검 청문회 윤 감추려는 진실 드러날 자리
7. 원희룡 모레 대표 출마선언 한동훈과 한날 출사표
8. 속보 유승민 국민의힘 전대 불출마 무의미한 도전
9. 당 대표 출마 준비중인 한동훈 전 비대위원장 응원 화환
10. 즐비하게 놓여져 있는 한동훈 응원 화환
11. 당 대표 출마 준비중인 한동훈 전 비대위원장 응원화환
12. 박찬대 해병대원 특검법 또다시 거부한다

In [14]:
from bs4 import BeautifulSoup
import requests as rq
import re
import pandas as pd

# HTTP 요청에서 사용될 헤더 정보를 설정합니다
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}




url = 'https://news.daum.net/politics#1'
    # urls.append(url)
print(url)
r = rq.get(url, headers=headers)
html = r.content
soup = BeautifulSoup(html, 'html.parser')

# lines = soup.select('body > div > main > section > div > div > ul > li > div > div > strong > a')
# lines = soup.find('strong', {'class' : 'tit_g'}).findAll('strong')
# lines = soup.find('strong', {'class' : 'tit_g'}).findAll('strong')

lines = soup.select('.list_realtime > li')
print(lines)
# 한글로만 가져올때
# for i, tag in enumerate(lines, start=1):
#     text = tag.get_text().strip().replace(',', '')
#     matches = re.findall('[가-힣0-9]+', text)
#     print(f'{i}. {" ".join(matches)}')

https://news.daum.net/politics#1
[]


Task3_0620. 앞에서 출력한 기사 리스트를 pandas 데이터프레임으로 변환 후 csv 파일로 저장 후 다시 불러오세요

In [35]:
import requests
from bs4 import BeautifulSoup
import re

for j in range(1, 11):
    #https://news.daum.net/breakingnews/politics?page=1
    url = f'https://news.daum.net/breakingnews/politics?page={j}'
    print(url)
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    lines = soup.find_all('a',class_="link_txt")
    lines = lines[7:-21]
    for i, tag in enumerate(lines, 1):
        text = tag.get_text().strip().replace(',', '')
        matches = re.findall('[가-힣0-9]+', text)
        print(f"{i}. {' '.join(matches)}")
    print(f'페이지수: {j}')

#         df = pd.DataFrame(matches)
# df
# # csv로 저장
# df.to_csv('news.csv', index=False)
# df = pd.read_csv('news.csv')
# df

https://news.daum.net/breakingnews/politics?page=1
1. 추경호 원내대표 이공계지원 특별법 개정 토론회 참석
2. 이공계지원 특별법 개정 토론회 참석한 추경호 원내대표
3. 국기에 경례하는 추경호 원내대표
4. 유승민 전당대회 불출마 선언 무의미한 도전
5. 자유북한운동연합 경기도 파주에서 대북 전단 살포
6. 자유북한운동연합 경기도 파주에서 대북 전단 살포
7. 박찬대 채해병 순직 1주기 특검법 통과 거부권 포기해야
8. 인터뷰 7년 갇혀있던 갈비사자 이젠 짝꿍도 생겼어요
9. 보훈부 보훈위탁병원 비상진료체계 현장 점검
10. 제73회 서울시문화상 추천 접수
11. 발언하는 박찬대 원내대표
12. 탈북민단체 대북전단 30만장 살포
13. 연금 전문가들 한국 연금체계 지속 불가능
14. 미 어떤 지원도 환영 미군 대응 태세 평가
15. 생각에 잠긴 이재명 대표
페이지수: 1
https://news.daum.net/breakingnews/politics?page=2
1. 발언하는 이재명 대표
2. 발언하는 이재명 대표
3. 게시판 한화에어로 6 25 참전용사 맞춤형 신발 헌정
4. 더불어민주당 최고위원회의
5. 생각에 잠긴 이재명 대표
6. 윤상현 일주일 전 나 돕겠다던 원희룡 어제는 죄송하다고
7. 더불어민주당 최고위원회의
8. 최고위 발언하는 이재명 대표
9. 발언하는 이재명 대표
10. 민주당 최고위원회의
11. 발언하는 이재명 대표
12. 건희권익위로 개명하라 박찬대 김 여사 면죄부 주려고 청탁금지법 왜곡 해석
13. 발언하는 이재명 대표
14. 최고위 발언하는 이재명 대표
15. 최고위 발언하는 이재명 대표
페이지수: 2
https://news.daum.net/breakingnews/politics?page=3
1. 최고위 발언하는 이재명 대표
2. 한동훈의 러닝메이트 누구 친윤계는 당헌당규 셀프 개정 가능성으로 압박 이런정치
3. 김성태 나경원은 온실 속 민들레 들꽃 되기도 받기도 어정쩡
4. 포토 최고위 발언하

In [43]:
# 강사님 버전
import re
import pandas as pd

soup = BeautifulSoup(html, 'html.parser')

lines = soup.select('body > div > main > section > div > div > ul > li > strong > a')
print(lines)

from bs4 import BeautifulSoup
import requests as rq
import re
import pandas as pd

# HTTP 요청에서 사용될 헤더 정보를 설정합니다
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}




url = 'https://news.daum.net/politics#1'
    # urls.append(url)
# print(url)
r = rq.get(url, headers=headers)
html = r.content
soup = BeautifulSoup(html, 'html.parser')

lines = soup.select('body > div > main > section > div > div > ul > li > div > div > strong > a')
# print(lines)

results = []
for i, tag in enumerate(lines, start=1):
    text = tag.get_text().strip().replace(',', '')
    matches = re.findall('[가-힣0-9]+', text)
    result = ' '.join(matches)
    results.append({'Index':i, 'Text':result})
    
print(results)
df = pd.DataFrame(results)
# df
df.to_csv('news.csv', index=False)
df = pd.read_csv('news.csv')
df

[<a class="link_txt" href="https://v.daum.net/v/20240621094039900">대통령실 “우크라이나에 대한 무기 지원 문제 재검토”</a>, <a class="link_txt" href="https://v.daum.net/v/20240621093551704">원희룡, 출마 발표 전날 용산 찾아…尹대통령 "존중한다"</a>, <a class="link_txt" href="https://v.daum.net/v/20240621093334630">[정치쇼] 김재섭 "천하제일 '비윤' 호소대회…원희룡·尹 모종의 일 있었을 것"</a>, <a class="link_txt" href="https://v.daum.net/v/20240621093344643">북러 고강도 군사 밀착에 한미도 면밀 대응…외교장관 긴급 통화</a>, <a class="link_txt" href="https://v.daum.net/v/20240621093230576">“영부인께 로비하고 싶습니다”… 권익위 ‘조롱’ 잇따라</a>, <a class="link_txt" href="https://v.daum.net/v/20240621093225570">북·러 ‘군사 협력’ 범위 불분명…“생각보다 더 장기적·더 위험”</a>, <a class="link_txt" href="https://v.daum.net/v/20240621092907420">김현기 서울시의장, 몽골 울란바토르에 소방차 10대 전달</a>, <a class="link_txt" href="https://v.daum.net/v/20240621092848408">북한, '6·25 미제 반대 투쟁의 날' 앞두고…"원수 격멸 의지 굳건"</a>, <a class="link_txt" href="https://v.daum.net/v/20240621092703349">대통령실 '야 단독소집' 운영위 불참키로…야, 증인 채택할듯</a>, <a class="link_txt" href="https://v.daum.ne

,Index,Text
0,1,특검 청문회 앞둔 민주 사실상 윤 특검 책임 반드시 물을 것 종합
1,2,한민수 재개발 시행사 용적률 로 원주민 정착률 법안프리즘
2,3,시선집중 박정훈 대령 측 이첩 기록 회수 지시했을 것 이시원은 기록 회수 임기훈은 ...
3,4,최재성 어대명 이인영 당권도전 고민 중 김성태 삼족을 멸할 것
4,5,한동훈의 러닝메이트 누구 친윤계는 당헌당규 셀프 개정 가능성으로 압박 이런정치
5,6,인터뷰 7년 갇혀있던 갈비사자 이젠 짝꿍도 생겼어요
6,7,한동훈의 러닝메이트 누구 친윤계는 당헌당규 셀프 개정 가능성으로 압박
7,8,남인순 더불어민주당 의원 의정 갈등 저출생 연금개혁 견해는
8,9,김재섭 천하제일 비윤 호소대회 원희룡 모종의 일 있었을 것
9,10,한미동맹엔 없지만 북중동맹에 있는 것 새로운 북러 조약에도
